In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from Bio import Entrez
from tqdm import tqdm

In [23]:
data = pd.read_csv (r'data/SFARI-Genes.csv')
cols = data.columns
N, D = data.shape
print(cols)

Index(['status', 'gene-symbol', 'gene-name', 'ensembl-id', 'chromosome',
       'genetic-category', 'gene-score', 'syndromic', 'number-of-reports'],
      dtype='object')


# Task 1

In [24]:
gene_symbols = data['gene-symbol'].unique()
print(gene_symbols.size)

1023


In [25]:
def save_to_txt_file(contents, filename):
    DIR = os.getcwd() + "/results/" + filename
    
    with open(DIR, "w") as f:
        f.write(contents)

In [ ]:
Entrez.email = "s1803764@ed.ac.uk"
output = ""

for i in tqdm(range(gene_symbols.size)):
    g = gene_symbols[i]
    handle = Entrez.esearch(db="gene", retmax=10, term=g + "[sym] homo sapiens[Organism]")
    record = Entrez.read(handle)
    output += g + ": " + ", ".join(str(x) for x in record['IdList'])
    output += "\n"
    
save_to_txt_file(output, "task2-1-eid.txt")

# Task 2

In [6]:
gene2go = np.loadtxt(os.getcwd() + "/data/gene2go", dtype=str, delimiter="\n")

In [7]:
gene2go.shape

(2084178,)

In [3]:
#task2_data = [line[12:22] for line in gene2go]
#print(task2_data)

task2_data = np.genfromtxt(gene2go, delimiter="\t")
np.save(os.getcwd() + "/data/task2.npy", task2_data, allow_pickle=True)

NameError: name 'gene2go' is not defined

In [4]:
len("3702\t814629\tGO:0005634")

22